In [1]:
import pandas as pd
import numpy as np
import time
import lxml
from concurrent.futures.thread import ThreadPoolExecutor
import asyncio
import nest_asyncio
nest_asyncio.apply()

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import exceptions as e

from selenium.webdriver.chrome.options import Options  



In [3]:
# init cn
global cns
global cns_result
global cns_fail
cns = ['신한디에스', '메가존', '똥멍충이']
cns_result = []
cns_fail = 0

In [3]:
def webdriver_init(dw= '../webdriver/chromedriver'):
    chrome_options = Options()  
    chrome_options.add_argument("--headless")
    return Chrome(dw, options=chrome_options)

In [4]:
def base_site(browser, delay= 5, url= 'https://kreditjob.com/company'):
    
    browser.get(url)
    browser.implicitly_wait(delay)

In [5]:
def cn_check(browser, cn, delay= 5):
    flag = True
    browser.find_element_by_class_name('search-query').send_keys(cn)
    browser.implicitly_wait(0.1)
    browser.find_element_by_class_name('search-query').send_keys(Keys.ENTER)
    
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'company-name')))
    except e.TimeoutException:
        flag = False
    return flag
    

In [32]:
def get_cn_info(browser, cn):
    result = [cn]
    
    area = ''
    industry = ''
    cn_type = ''
    salary = ''
    emp_count = ''
    on_bs_years = ''
    
    try:
        common_datas = browser.find_elements_by_xpath("//span[@class='cat_text']")
        area, industry, cn_type = common_datas[0].text, common_datas[1].text, common_datas[2].text
    except Exception as e:
        pass
        
    try:
        salary = browser.find_element_by_class_name('salary').text
    except Exception as e:
        pass

    try:
        info_datas = browser.find_elements_by_xpath("//span[@class='company-info-data']")
        emp_count, on_bs_years = info_datas[0].text, info_datas[1].text
    except Exception as e:
        pass
        
#     ratios = browser.find_elements_by_xpath("//span[@class='company-info-ratio']")
#     join, leave = ratios[0].text, ratios[1].text
#     sales_data = browser.find_elements_by_xpath("//span[@class='sales-data']")
#     revenue, revenue_per_emp = sales_data[0].text, sales_data[1].text

    result.append(area)
    result.append(industry)
    result.append(cn_type)
    result.append(salary)
    result.append(emp_count)
    result.append(on_bs_years)
#     result.append(join)
#     result.append(leave)
#     result.append(revenue)
#     result.append(revenue_per_emp)
    
    return result

In [7]:
def execution_all(cn):
    global cns_fail

    browser = webdriver_init()
    base_site(browser, 0.1)
    
    if cn_check(browser, cn):
        result = get_cn_info(browser, cn)
        cns_result.append(result)
    else:
        cns_fail += 1
    
    browser.close()
#     print('finish: {}'.format(cn))

In [7]:
def dataset(data):
    df_columns = ['회사명','지역','산업군','type','연봉','인원','업력']
#     df_columns = ['회사명','지역','산업군','type','연봉','인원','업력','입사율','퇴사율','매출액','1인당매출액']
    df = pd.DataFrame(data, columns=df_columns)
    return df

In [6]:
def save_csv(df):
    now = time.localtime()
    now = time.strftime("%Y%m%d%H%M%S", now)
    file_name = '../data/result/' + now + '.csv'
    df.to_csv(file_name,'\t',mode='w')

In [5]:
def run_cn_checker_async(thread_workers_count, cns):
    loop = asyncio.get_running_loop()
    executor = ThreadPoolExecutor(thread_workers_count)
    
    for cn in cns:
        loop.run_in_executor(executor, execution_all, cn)

        
    loop.run_until_complete(asyncio.gather(*asyncio.all_tasks(loop)))    